#**Study Case: Evaluating Quality from Projects of MIT App Inventor**




---



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import scipy.cluster.hierarchy as hc
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.cluster import AgglomerativeClustering

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file = "/content/drive/My Drive/appinventor_analysis/dataset.csv"

**1) Statidistic data extracted from dataset:**


---



In [ ]:
df = pd.read_csv(file, sep=';')

In [ ]:
df.describe()

,AppVersionCode,NumberOfScreens,NumberOfTotalBlocks,NumberOfDefinedFunctionalBlocks,NumberOfUsedFunctionalBlocks,NumberOfEventsBlocks,NumberOfConditionalBlocks,NumberOfLoopsBlocks,NumberOfGlobalVariables,NumberOfLocalVariables,NumberOfUserInterfaceBlocks,NumberOfLayoutBlocks,NumberOfMediaBlocks,NumberOfDrawingAnimationBlocks,NumberOfMapsBlocks,NumberOfChartsBlocks,NumberOfSensorsBlocks,NumberOfSocialBlocks,NumberOfStorageBlocks,NumberOfConnectivityBlocks,NumberOfLegoBlocks,NumberOfExperimentalBlocks,NumberOfScreensErrors,PercentageDuplicateBlocks,NameOfVariablesErrors,NameOfFunctionsErrors,NumberOfParamatersErrors,VariablesNotUsed,PercentageComments,MagicNumbers,NumberOfFunctionsWithTooManyBlocks,NumberOfIfBlocksTooManyNested,MinCiclomaticComplexity,MaxCiclomaticComplexity,AvgCiclomaticComplexity,MinCognitiveComplexity,MaxCognitiveComplexity,AvgCognitiveComplexity,NumberOfDivisionByZero,NumberOfInfiniteLoops,NumberOfVariablesNotInitialized,NumberOfIndexOutOfRange,NumberOfDefinedFunctionalBlocksNotUsed,NumberOfFunctionalBlocksWithoutReturn,NumberOfWhileLoopsWithoutCondition,NumberOfVariablesInConditionNotUpdated,NumberOfCamelCaseVariables,NumberOfCamelCaseDefinedFunctionalBlocks,NumberOfPascalCaseVariables,NumberOfPascalCaseDefinedFunctionalBlocks,NumberOfSnakeCaseVariables,NumberOfSnakeCaseDefinedFunctionalBlocks,NumberOfNoneCaseVariables,NumberOfNoneCaseDefinedFunctionalBlocks
count,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.00000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.0,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000
mean,1.284844,1.153300,128.620649,1.447999,9.941778,8.266837,3.946633,0.266990,3.463316,0.30812,54.414130,2.488878,5.535950,27.232053,0.171387,0.0,5.503447,0.533511,4.923742,5.066018,0.143052,0.313932,0.000794,1.055744,0.308817,0.021836,0.050111,0.267784,0.365723,1.449992,0.128069,0.587440,0.625202,0.917856,0.724913,0.812845,1.490564,1.048406,0.001942,0.005357,0.039393,0.041855,0.101917,0.001905,0.000929,0.731421,0.522932,0.384020,0.315869,0.288812,0.586818,0.208373,2.345817,0.566794
std,8.437006,0.756459,138.737149,2.617231,13.532433,10.069302,5.811005,0.980861,4.889437,1.11695,128.152058,16.835835,25.883707,58.002135,8.137110,0.0,14.792882,6.055018,23.418029,19.975531,2.904550,4.747960,0.028175,6.495454,1.118109,0.259472,0.263312,1.351070,1.796877,4.776120,0.490103,1.985079,0.826539,1.267157,0.930928,1.818769,3.008281,2.058346,0.077691,0.171663,0.501245,0.788914,0.546901,0.198268,0.056611,1.338948,1.540991,1.187401,1.223953,1.184697,2.231482,0.929029,3.649413,1.550008
min,0.000000,1.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,85.000000,0.000000,3.000000,4.000000,1.000000,0.000000,1.000000,0.00000,15.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

**2) Calculating Technical Debt from the projects:**


---



In [ ]:
factorLevel1 = 0.1
factorLevel2 = 1
factorLevel3 = 5
factorLevel4 = 20

In [ ]:
timePerBlock = 86.4

In [ ]:
technicalDebt = [0] * 215244

**Code Smells & Bugs Level 1**

---



**M25**: NameOfVariableError

In [ ]:
for index, row in df.iterrows():
  if(row.NameOfVariablesErrors > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NameOfVariablesErrors * timePerBlock) * factorLevel1


**M26**: NameOfFunctionError

In [ ]:
for index, row in df.iterrows():
  if(row.NameOfFunctionsErrors > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NameOfFunctionsErrors * timePerBlock) * factorLevel1

**M45**: PercentageComments

In [ ]:
maximumPercentageOfComments = 0.3

for index, row in df.iterrows():
  if(row.PercentageComments > maximumPercentageOfComments):
    percentageOfCommentsToDelete = row.PercentageComments - maximumPercentageOfComments
    technicalDebt[index] = technicalDebt[index] + (percentageOfCommentsToDelete * timePerBlock) * factorLevel1

**Code Smells & Bugs Level 2**

---



**M23**: NumberOfDefinedFunctionalBlocksNotUsed

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfDefinedFunctionalBlocksNotUsed > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfDefinedFunctionalBlocksNotUsed * timePerBlock) * factorLevel2

**M35**: VariablesNotUsed

In [ ]:
for index, row in df.iterrows():
  if(row.VariablesNotUsed > 1):
    technicalDebt[index] = technicalDebt[index] + (row.VariablesNotUsed * timePerBlock) * factorLevel2

**M43**: NumberOfParametersErrors

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfParamatersErrors > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfParamatersErrors * timePerBlock) * factorLevel2

**M48**: NumberOfVariableNotInitialized

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfVariablesNotInitialized > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfVariablesNotInitialized * timePerBlock) * factorLevel2

**M49**: NumberOfIndexOutOfRange

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfIndexOutOfRange > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfIndexOutOfRange * timePerBlock) * factorLevel2

**M50**: NumberOfVariablesInConditionNotUpdated

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfVariablesInConditionNotUpdated > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfVariablesInConditionNotUpdated * timePerBlock) * factorLevel2

**M51**: NumberOfFunctionalBlocksWithoutReturn

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfFunctionalBlocksWithoutReturn > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfFunctionalBlocksWithoutReturn * timePerBlock) * factorLevel2

**Code Smells & Bugs Level 3**

---



**M42**: MagicNumbers

In [ ]:
for index, row in df.iterrows():
  if(row.MagicNumbers > 1):
    technicalDebt[index] = technicalDebt[index] + (row.MagicNumbers * timePerBlock) * factorLevel3

**M44**: NumberOfIfBlocksTooManyNested

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfIfBlocksTooManyNested > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfIfBlocksTooManyNested * timePerBlock) * factorLevel3

**M46**: NumberOfDivisionByZero

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfDivisionByZero > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfDivisionByZero * timePerBlock) * factorLevel3

**M47**: NumberOfInfiniteLoops

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfInfiniteLoops > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfInfiniteLoops * timePerBlock) * factorLevel3

**M52**: NumberOfWhileLoopsWithoutCondition

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfWhileLoopsWithoutCondition > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfWhileLoopsWithoutCondition * timePerBlock) * factorLevel3

**Code Smells Level 4**


---



**M22**: PercentageDuplicateBlocks

In [ ]:
maximumPercentageOfDuplicateBlocks = 0.2

for index, row in df.iterrows():
  if(row.PercentageDuplicateBlocks > maximumPercentageOfDuplicateBlocks):
    percentageOfDuplicateBlocksToDelete = row.PercentageDuplicateBlocks - maximumPercentageOfDuplicateBlocks
    technicalDebt[index] = technicalDebt[index] + (percentageOfDuplicateBlocksToDelete * timePerBlock) * factorLevel4

**M24**: NumberOfFunctionsWithTooManyBlocks

In [ ]:
for index, row in df.iterrows():
  if(row.NumberOfFunctionsWithTooManyBlocks > 1):
    technicalDebt[index] = technicalDebt[index] + (row.NumberOfFunctionsWithTooManyBlocks * timePerBlock) * factorLevel3

**Adding the new column "TechnicalDebt" to the dataset:**


---



In [ ]:
df['TechnicalDebt'] = technicalDebt

In [ ]:
df.describe()

,AppVersionCode,NumberOfScreens,NumberOfTotalBlocks,NumberOfDefinedFunctionalBlocks,NumberOfUsedFunctionalBlocks,NumberOfEventsBlocks,NumberOfConditionalBlocks,NumberOfLoopsBlocks,NumberOfGlobalVariables,NumberOfLocalVariables,NumberOfUserInterfaceBlocks,NumberOfLayoutBlocks,NumberOfMediaBlocks,NumberOfDrawingAnimationBlocks,NumberOfMapsBlocks,NumberOfChartsBlocks,NumberOfSensorsBlocks,NumberOfSocialBlocks,NumberOfStorageBlocks,NumberOfConnectivityBlocks,NumberOfLegoBlocks,NumberOfExperimentalBlocks,NumberOfScreensErrors,PercentageDuplicateBlocks,NameOfVariablesErrors,NameOfFunctionsErrors,NumberOfParamatersErrors,VariablesNotUsed,PercentageComments,MagicNumbers,NumberOfFunctionsWithTooManyBlocks,NumberOfIfBlocksTooManyNested,MinCiclomaticComplexity,MaxCiclomaticComplexity,AvgCiclomaticComplexity,MinCognitiveComplexity,MaxCognitiveComplexity,AvgCognitiveComplexity,NumberOfDivisionByZero,NumberOfInfiniteLoops,NumberOfVariablesNotInitialized,NumberOfIndexOutOfRange,NumberOfDefinedFunctionalBlocksNotUsed,NumberOfFunctionalBlocksWithoutReturn,NumberOfWhileLoopsWithoutCondition,NumberOfVariablesInConditionNotUpdated,NumberOfCamelCaseVariables,NumberOfCamelCaseDefinedFunctionalBlocks,NumberOfPascalCaseVariables,NumberOfPascalCaseDefinedFunctionalBlocks,NumberOfSnakeCaseVariables,NumberOfSnakeCaseDefinedFunctionalBlocks,NumberOfNoneCaseVariables,NumberOfNoneCaseDefinedFunctionalBlocks,TechnicalDebt
count,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.00000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.0,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000
mean,1.284844,1.153300,128.620649,1.447999,9.941778,8.266837,3.946633,0.266990,3.463316,0.30812,54.414130,2.488878,5.535950,27.232053,0.171387,0.0,5.503447,0.533511,4.923742,5.066018,0.143052,0.313932,0.000794,1.055744,0.308817,0.021836,0.050111,0.267784,0.365723,1.449992,0.128069,0.587440,0.625202,0.917856,0.724913,0.812845,1.490564,1.048406,0.001942,0.005357,0.039393,0.041855,0.101917,0.001905,0.000929,0.731421,0.522932,0.384020,0.315869,0.288812,0.586818,0.208373,2.345817,0.566794,2735.341902
std,8.437006,0.756459,138.737149,2.617231,13.532433,10.069302,5.811005,0.980861,4.889437,1.11695,128.152058,16.835835,25.883707,58.002135,8.137110,0.0,14.792882,6.055018,23.418029,19.975531,2.904550,4.747960,0.028175,6.495454,1.118109,0.259472,0.263312,1.351070,1.796877,4.776120,0.490103,1.985079,0.826539,1.267157,0.930928,1.818769,3.008281,2.058346,0.077691,0.171663,0.501245,0.788914,0.546901,0.198268,0.056611,1.338948,1.540991,1.187401,1.223953,1.184697,2.231482,0.929029,3.649413,1.550008,11519.085121
min,0.000000,1.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,85.000000,0.000000,3.000000,4.000000,1.000000,0.000000,1.000000,0.00000,15.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

**3) Calculating Maintainability from the projects:**


---



In [ ]:
maintainability = [0] * 215244

In [ ]:
for index, row in df.iterrows():
  investedTime = df.NumberOfTotalBlocks[index] * timePerBlock
  technicalDebtRatio = df.TechnicalDebt[index] / investedTime

  if technicalDebtRatio <= 0.05:
    maintainability[index] = 'A'
  elif technicalDebtRatio >= 0.06 and technicalDebtRatio <= 0.1:
    maintainability[index] = 'B'
  elif technicalDebtRatio >= 0.11 and technicalDebtRatio <= 0.2:
    maintainability[index] = 'C'
  elif technicalDebtRatio >= 0.21 and technicalDebtRatio <= 0.5:
    maintainability[index] = 'D'
  else:
    maintainability[index] = 'E'

**Adding the new column "Time" to the dataset:**


---



In [ ]:
df['Maintainability'] = maintainability

In [ ]:
df.describe()

,AppVersionCode,NumberOfScreens,NumberOfTotalBlocks,NumberOfDefinedFunctionalBlocks,NumberOfUsedFunctionalBlocks,NumberOfEventsBlocks,NumberOfConditionalBlocks,NumberOfLoopsBlocks,NumberOfGlobalVariables,NumberOfLocalVariables,NumberOfUserInterfaceBlocks,NumberOfLayoutBlocks,NumberOfMediaBlocks,NumberOfDrawingAnimationBlocks,NumberOfMapsBlocks,NumberOfChartsBlocks,NumberOfSensorsBlocks,NumberOfSocialBlocks,NumberOfStorageBlocks,NumberOfConnectivityBlocks,NumberOfLegoBlocks,NumberOfExperimentalBlocks,NumberOfScreensErrors,PercentageDuplicateBlocks,NameOfVariablesErrors,NameOfFunctionsErrors,NumberOfParamatersErrors,VariablesNotUsed,PercentageComments,MagicNumbers,NumberOfFunctionsWithTooManyBlocks,NumberOfIfBlocksTooManyNested,MinCiclomaticComplexity,MaxCiclomaticComplexity,AvgCiclomaticComplexity,MinCognitiveComplexity,MaxCognitiveComplexity,AvgCognitiveComplexity,NumberOfDivisionByZero,NumberOfInfiniteLoops,NumberOfVariablesNotInitialized,NumberOfIndexOutOfRange,NumberOfDefinedFunctionalBlocksNotUsed,NumberOfFunctionalBlocksWithoutReturn,NumberOfWhileLoopsWithoutCondition,NumberOfVariablesInConditionNotUpdated,NumberOfCamelCaseVariables,NumberOfCamelCaseDefinedFunctionalBlocks,NumberOfPascalCaseVariables,NumberOfPascalCaseDefinedFunctionalBlocks,NumberOfSnakeCaseVariables,NumberOfSnakeCaseDefinedFunctionalBlocks,NumberOfNoneCaseVariables,NumberOfNoneCaseDefinedFunctionalBlocks,TechnicalDebt
count,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.00000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.0,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000,215244.000000
mean,1.284844,1.153300,128.620649,1.447999,9.941778,8.266837,3.946633,0.266990,3.463316,0.30812,54.414130,2.488878,5.535950,27.232053,0.171387,0.0,5.503447,0.533511,4.923742,5.066018,0.143052,0.313932,0.000794,1.055744,0.308817,0.021836,0.050111,0.267784,0.365723,1.449992,0.128069,0.587440,0.625202,0.917856,0.724913,0.812845,1.490564,1.048406,0.001942,0.005357,0.039393,0.041855,0.101917,0.001905,0.000929,0.731421,0.522932,0.384020,0.315869,0.288812,0.586818,0.208373,2.345817,0.566794,2735.341902
std,8.437006,0.756459,138.737149,2.617231,13.532433,10.069302,5.811005,0.980861,4.889437,1.11695,128.152058,16.835835,25.883707,58.002135,8.137110,0.0,14.792882,6.055018,23.418029,19.975531,2.904550,4.747960,0.028175,6.495454,1.118109,0.259472,0.263312,1.351070,1.796877,4.776120,0.490103,1.985079,0.826539,1.267157,0.930928,1.818769,3.008281,2.058346,0.077691,0.171663,0.501245,0.788914,0.546901,0.198268,0.056611,1.338948,1.540991,1.187401,1.223953,1.184697,2.231482,0.929029,3.649413,1.550008,11519.085121
min,0.000000,1.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,85.000000,0.000000,3.000000,4.000000,1.000000,0.000000,1.000000,0.00000,15.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

**Projects with Maintainability = A**


---



In [ ]:
projectsWithCategoryA = (df.Maintainability == 'A').sum()
percentageProjectsWithCategoryA = 100 * float(projectsWithCategoryA)/float(215244)

In [ ]:
print(projectsWithCategoryA)
print(str(percentageProjectsWithCategoryA) + "%")

150554
69.94573600193269%


**Projects with Maintainability = B**


---



In [ ]:
projectsWithCategoryB = (df.Maintainability == 'B').sum()
percentageProjectsWithCategoryB = 100 * float(projectsWithCategoryB)/float(215244)

In [ ]:
print(projectsWithCategoryB)
print(str(percentageProjectsWithCategoryB) + "%")

9349
4.343442790507517%


**Projects with Maintainability = C**


---



In [ ]:
projectsWithCategoryC = (df.Maintainability == 'C').sum()
percentageProjectsWithCategoryC = 100 * float(projectsWithCategoryC)/float(215244)

In [ ]:
print(projectsWithCategoryC)
print(str(percentageProjectsWithCategoryC) + "%")

15518
7.209492482949583%


**Projects with Maintainability = D**


---



In [ ]:
projectsWithCategoryD = (df.Maintainability == 'D').sum()
percentageProjectsWithCategoryD = 100 * float(projectsWithCategoryD)/float(215244)

In [ ]:
print(projectsWithCategoryD)
print(str(percentageProjectsWithCategoryD) + "%")

16812
7.810670680715838%


**Projects with Maintainability = E**


---



In [ ]:
projectsWithCategoryE = (df.Maintainability == 'E').sum()
percentageProjectsWithCategoryE = 100 * float(projectsWithCategoryE)/float(215244)

In [ ]:
print(projectsWithCategoryE)
print(str(percentageProjectsWithCategoryE) + "%")

23011
10.690658043894372%


**Graphic by Maintainability Category:**


---



In [ ]:
df.groupby('Maintainability').agg({'AppVersionCode': 'mean', 'NumberOfTotalBlocks': 'mean', 'TechnicalDebt': 'mean', 'PercentageDuplicateBlocks': 'mean', 'NumberOfDefinedFunctionalBlocksNotUsed': 'mean', 'NumberOfFunctionsWithTooManyBlocks' : 'mean', 'NameOfVariablesErrors' : 'mean', 'NameOfFunctionsErrors' : 'mean', 'VariablesNotUsed' : 'mean', 'MagicNumbers' : 'mean', 'NumberOfParamatersErrors' : 'mean', 'NumberOfIfBlocksTooManyNested' : 'mean', 'PercentageComments' : 'mean', 'NumberOfDivisionByZero' : 'mean', 'NumberOfInfiniteLoops' : 'mean', 'NumberOfVariablesNotInitialized' : 'mean', 'NumberOfIndexOutOfRange' : 'mean', 'NumberOfVariablesInConditionNotUpdated' : 'mean', 'NumberOfFunctionalBlocksWithoutReturn' : 'mean', 'NumberOfWhileLoopsWithoutCondition' : 'mean'})

,AppVersionCode,NumberOfTotalBlocks,TechnicalDebt,PercentageDuplicateBlocks,NumberOfDefinedFunctionalBlocksNotUsed,NumberOfFunctionsWithTooManyBlocks,NameOfVariablesErrors,NameOfFunctionsErrors,VariablesNotUsed,MagicNumbers,NumberOfParamatersErrors,NumberOfIfBlocksTooManyNested,PercentageComments,NumberOfDivisionByZero,NumberOfInfiniteLoops,NumberOfVariablesNotInitialized,NumberOfIndexOutOfRange,NumberOfVariablesInConditionNotUpdated,NumberOfFunctionalBlocksWithoutReturn,NumberOfWhileLoopsWithoutCondition
Maintainability,,,,,,,,,,,,,,,,,,,,
A,1.293569,124.134397,74.438812,0.000051,0.070261,0.096876,0.296731,0.015795,0.169388,0.062051,0.047113,0.100223,0.380059,0.000452,0.001574,0.027160,0.021022,0.717862,0.001182,0.000558
B,1.249331,177.312119,1201.714653,0.006006,0.153278,0.458124,0.428602,0.045352,0.808108,0.810461,0.057011,1.255535,0.453564,0.006204,0.013156,0.122152,0.159803,0.971869,0.001177,0.001284
C,1.189329,132.448125,1715.997783,0.019536,0.139000,0.200348,0.367058,0.050135,0.518624,2.016110,0.085191,1.581132,0.324173,0.008957,0.020750,0.058899,0.090411,0.924604,0.001096,0.001160
D,1.334582,125.745182,3510.564072,0.068568,0.172317,0.138591,0.406912,0.027480,0.347074,5.471389,0.052106,2.213597,0.254479,0.004045,0.009576,0.043778,0.049310,0.631216,0.001071,0.000952
E,1.270262,137.709965,20888.948084,9.809346,0.211725,0.141628,0.228282,0.028595,0.464952,7.470862,0.041806,1.645517,0.345537,0.003694,0.013472,0.069445,0.092043,0.665377,0.008083,0.003042
